In [ ]:
https://github.com/Kazuhito00/hand-gesture-recognition-using-mediapipe/tree/main

In [ ]:
import csv

import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split

RANDOM_SEED = 42

# 경로 지정

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pwd

/content


In [ ]:
x_dataset_path = '/content/drive/MyDrive/KHUDA3rd 3차/0724/final_dataset.csv'
y_dataset_path = '/content/drive/MyDrive/KHUDA3rd 3차/0724/final_y_dataset.csv'
model_save_path = '/content/drive/MyDrive/KHUDA3rd 3차/0724/keypoint_classifier_0725_lstm.hdf5'

# 분류수 설정

In [ ]:
NUM_CLASSES = 12

# 학습 데이터 불러오기

In [ ]:
X_dataset = np.loadtxt(x_dataset_path, delimiter=',', dtype='float32', usecols=list(range(3, (21 * 2) + 3)), skiprows=True)

In [ ]:
X_dataset = X_dataset.reshape(1064, 56, 42)
# X_dataset = X_dataset.reshape(1064, 42, 56, 1)

In [ ]:
y_dataset = np.loadtxt(y_dataset_path, delimiter=',', dtype='int')

In [ ]:
y_dataset.shape

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_dataset, y_dataset, train_size=0.75, random_state=RANDOM_SEED)

# 모델 구축

In [ ]:
# model = tf.keras.models.Sequential([
#     tf.keras.layers.Input((21 * 2 * 56, )),
#     tf.keras.layers.Dropout(0.2),
#     tf.keras.layers.Dense(20, activation='relu'),
#     tf.keras.layers.Dropout(0.4),
#     tf.keras.layers.Dense(10, activation='relu'),
#     tf.keras.layers.Dense(NUM_CLASSES, activation='softmax')
# ])

model = tf.keras.Sequential([
  tf.keras.layers.LSTM(64, activation='relu', input_shape=((56, 42, ))),
  tf.keras.layers.Dropout(0.5),
  tf.keras.layers.Dense(10, activation='relu'),
  tf.keras.layers.Dense(NUM_CLASSES, activation='softmax')
])


# model = tf.keras.models.Sequential([
#     tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(42, 56, 1)),
#     tf.keras.layers.MaxPooling2D(2,2),
#     tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
#     tf.keras.layers.MaxPooling2D(2,2),
#     tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
#     tf.keras.layers.MaxPooling2D(2,2),
#     tf.keras.layers.Flatten(),
#     tf.keras.layers.Dense(512, activation='relu'),
#     tf.keras.layers.Dense(12, activation='softmax')
# ])

In [ ]:
model.summary()  # tf.keras.utils.plot_model(model, show_shapes=True)

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_3 (Conv2D)           (None, 40, 54, 16)        160       
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 20, 27, 16)       0         
 2D)                                                             
                                                                 
 conv2d_4 (Conv2D)           (None, 18, 25, 32)        4640      
                                                                 
 max_pooling2d_4 (MaxPooling  (None, 9, 12, 32)        0         
 2D)                                                             
                                                                 
 conv2d_5 (Conv2D)           (None, 7, 10, 64)         18496     
                                                                 
 max_pooling2d_5 (MaxPooling  (None, 3, 5, 64)        

In [ ]:
# 모델 체크포인트 콜백
cp_callback = tf.keras.callbacks.ModelCheckpoint(
    model_save_path, verbose=1, save_weights_only=False)
# 중간에 멈추는 경우 콜백
es_callback = tf.keras.callbacks.EarlyStopping(patience=20, verbose=1)

In [ ]:
#  model compile
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

# model train

In [ ]:
model.fit(
    X_train,
    y_train,
    epochs=1000,
    batch_size=128,
    validation_data=(X_test, y_test),
    callbacks=[cp_callback, es_callback]
)

Epoch 1/1000
7/7 [==============================] - ETA: 0s - loss: 2.4340 - accuracy: 0.1303  
Epoch 1: saving model to /content/drive/MyDrive/KHUDA3rd 3차/0724/keypoint_classifier_0725_lstm.hdf5
7/7 [==============================] - 12s 146ms/step - loss: 2.4340 - accuracy: 0.1303 - val_loss: 2.2726 - val_accuracy: 0.1842
Epoch 2/1000
1/7 [===>..........................] - ETA: 0s - loss: 2.2274 - accuracy: 0.2188
Epoch 2: saving model to /content/drive/MyDrive/KHUDA3rd 3차/0724/keypoint_classifier_0725_lstm.hdf5
7/7 [==============================] - 0s 22ms/step - loss: 2.1311 - accuracy: 0.2882 - val_loss: 1.9271 - val_accuracy: 0.2782
Epoch 3/1000
1/7 [===>..........................] - ETA: 0s - loss: 1.8582 - accuracy: 0.3984
Epoch 3: saving model to /content/drive/MyDrive/KHUDA3rd 3차/0724/keypoint_classifier_0725_lstm.hdf5
7/7 [==============================] - 0s 25ms/step - loss: 1.8150 - accuracy: 0.3759 - val_loss: 1.7020 - val_accuracy: 0.3383
Epoch 4/1000
1/7 [===>.....

In [ ]:
val_loss, val_acc = model.evaluate(X_test, y_test, batch_size=128)

3/3 [==============================] - 0s 5ms/step - loss: 0.2608 - accuracy: 0.9248


In [ ]:
model = tf.keras.models.load_model(model_save_path)

In [ ]:
predict_result = model.predict(np.array([X_test[0]]))
print(np.squeeze(predict_result))
print(np.argmax(np.squeeze(predict_result)))

In [ ]:
y_test

array([ 2, 11,  8,  6,  7,  1,  1,  1,  2,  8,  3, 11, 11,  0,  6, 11,  6,
        9,  2,  4,  7,  7,  5,  0,  9,  9,  7,  5,  4,  5, 10,  3,  8,  7,
        7,  5,  6,  1,  6,  3,  4, 11,  5,  6,  6,  7, 10,  8, 11, 10,  3,
        9,  6,  6,  0, 10,  2,  6,  7,  1,  3,  6,  6,  2,  8, 10,  7,  7,
        2,  6,  8,  6,  4,  6,  7,  1,  0,  0,  6,  6,  1,  7,  5,  7,  2,
        6,  6,  1,  2,  2,  0,  5,  1,  7,  0,  7,  0,  6,  8,  3,  9,  9,
        9,  3,  3,  5,  3,  4,  8,  5,  0,  9,  7,  4,  0,  3,  5,  3,  0,
       10,  5, 10,  2,  3,  0, 11,  2,  1,  1,  1,  2,  2,  9,  2,  2, 10,
        1,  7,  4,  2,  8,  8,  8,  6,  4,  4,  2,  2,  1,  9,  3,  0,  6,
        4,  4,  5,  4,  3,  0,  8,  1,  2, 11,  2,  0,  2, 10,  5, 11,  7,
        1,  5,  6,  0,  9, 10,  4,  5,  4,  6,  5,  6,  2,  0,  0,  0,  4,
        6, 11,  8,  7,  7,  1,  9,  1,  7,  1,  4,  8, 10,  7,  4,  0,  0,
       10, 10,  8,  3,  1,  2,  3, 11,  2,  2, 10,  2,  6,  7,  7,  7,  8,
       11, 11,  6,  9,  0

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, classification_report

def print_confusion_matrix(y_true, y_pred, report=True):
    labels = sorted(list(set(y_true)))
    cmx_data = confusion_matrix(y_true, y_pred, labels=labels)

    df_cmx = pd.DataFrame(cmx_data, index=labels, columns=labels)

    fig, ax = plt.subplots(figsize=(7, 6))
    sns.heatmap(df_cmx, annot=True, fmt='g' ,square=False)
    ax.set_ylim(len(set(y_true)), 0)
    plt.show()

    if report:
        print('Classification Report')
        print(classification_report(y_test, y_pred))

Y_pred = model.predict(X_test)
y_pred = np.argmax(Y_pred, axis=1)

print_confusion_matrix(y_test, y_pred)

# Tensorflow-Lite용 모델로 변환

In [ ]:
model.save(model_save_path, include_optimizer=False)

In [ ]:
tf.__version__

'2.12.0'

In [ ]:
tflite_save_path = '/content/drive/MyDrive/KHUDA3rd 3차/0724/keypoint_classifier_0726_cnn.tflite'
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.experimental_new_converter=True
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS,
tf.lite.OpsSet.SELECT_TF_OPS]
tflite_quantized_model = converter.convert()

open(tflite_save_path, 'wb').write(tflite_quantized_model)

529336

# 성능 test

In [ ]:
interpreter = tf.lite.Interpreter(model_path=tflite_save_path)
interpreter.allocate_tensors()

In [ ]:
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

In [ ]:
interpreter.set_tensor(input_details[0]['index'], np.array([X_test[0]]))

In [ ]:
%%time
interpreter.invoke()
tflite_results = interpreter.get_tensor(output_details[0]['index'])

CPU times: user 3.26 ms, sys: 945 µs, total: 4.2 ms
Wall time: 6.72 ms


In [ ]:
print(np.squeeze(tflite_results))
print(np.argmax(np.squeeze(tflite_results)))

[1.9128470e-06 1.2457697e-07 9.7453666e-01 1.5415688e-05 1.2349144e-03
 1.5118847e-07 2.2364464e-02 1.3173852e-06 1.2979107e-03 5.9402396e-06
 5.0541607e-04 3.5809608e-05]
2
